В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [1]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 73.1 MB/s eta 0:00:00


## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [2]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

[==================================================] 100.0% 128.1/128.1MB downloaded
Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

**Примеры датасетов из документации gensim**
1. glove-twitter-100

Датасет glove-twitter-100 представляет собой набор предварительно обученных векторов, созданных на основе 2 миллиардов твитов. Всего данные  составляют 27 миллиардов токенов или 1,2 миллиона слов, записанных без регистра. Размерность векторов равна 100, размерность словаря – 1193514. Датасет весит 387 МБ. Датасет может быть использован, например, для анализа тональности твитов пользователей или для определения тематик твитов: задачи кластеризации (или категоризации, если самостоятельно добавить таргет) по различным тематикам твитов.

2. word2vec-google-news-300

Датасет word2vec-google-news-300 представляет собой набор предварительно обученных векторов. Источником данных является электорнный ресурс Google News. В датасете используется приблизительно 100 миллиардов слов. Размерность векторов составляет 300, размерность словаря – 3000000. Датасет весит 1662 МБ. Датасет word2vec-google-news-300 может быть также использован для сентимент-анализа или, например, для создания вопросно-ответной системы по новостной тематики, на основе которой можно сделать чат-бота.

3. word2vec-ruscorpora-300

Датасет word2vec-ruscorpora-300 является набором векторов, полученных с помощью модели Word2vec скип-грамм (Continuous Skipgram). Источником для этого датасета послужил Национальный корпус русского языка (НКРЯ) в полном объеме: около 250 миллионов слов (но модель содержит 185 тыс. слов). Размерность векторов составляет 300, размер окна – 10, размер словаря 184973. Датасет весит 198 МБ. Поскольку данный датасет создан на основе НКРЯ, наиболее подходящая область применения – лингвистические исследования русского языка. Например, датасет может использоваться для исследования семантических полей слов (изучение концепта с помощью наиболее похожих слов).

**Базовые операции с векторами**

In [3]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [4]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

Слова, похожие на 'python':
  monty: 0.6886
  php: 0.5865
  perl: 0.5784
  cleese: 0.5447
  flipper: 0.5113


*Ваш ответ здесь*

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [5]:
# я использую датасет, собранный из твиттера, из описанных мной выше вариантов наборов данных, но меньшего объема
w2v_model_twitter = api.load('glove-twitter-50')

print(f"Размер словаря: {len(w2v_model_twitter.key_to_index)}")
print(f"Размерность векторов: {w2v_model_twitter.vector_size}")

[==================================================] 100.0% 199.5/199.5MB downloaded
Размер словаря: 1193514
Размерность векторов: 50


2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [21]:
# напишим функцию для поиска наиболее близких по вектору слов
def find_similar_words(word):

#добавим обработку ошибок на случай, если введеного слова нет в датасете
  try:
    # Получаем вектор слова
    vector = w2v_model_twitter[word]
    # находим 10 наиболее близких по вектору слов
    similar_words = w2v_model_twitter.most_similar(word, topn=10)
    print("Топ-10 наиболее близких по вектору слов к слову 'lol':")
    for synonym, score in similar_words:
      print(f"{synonym}: {score:.4f}")

  except KeyError:
    print(f"Слово '{word}' не найдено в словаре")


In [25]:
# примеh работы функцию
find_similar_words('lol')

Топ-10 наиболее близких по вектору слов к слову 'lol':
lmao: 0.9240
yeah: 0.9142
dude: 0.9132
wtf: 0.9009
aha: 0.8975
idk: 0.8962
shit: 0.8927
lmfao: 0.8925
thats: 0.8894
tho: 0.8787


In [28]:
# пример обработки ошибки
find_similar_words('кркап')

Слово 'кркап' не найдено в словаре


3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [16]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [17]:
# импортируем модель
from gensim.models import Word2Vec

# обучаем модель Word2Vec на датасете cooking_sentences
model = Word2Vec(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    sg=1
)

In [18]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [19]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [41]:
# для решения задания завернем предыдущий код в функцию
def find_synonym(word):
  try:
    similar = model.wv.most_similar(word, topn=5)
    print(f"\nСлова, похожие на '{word}':")
    for synonym, score in similar:
        print(f"{synonym}: {score:.4f}")
  except KeyError:
    print(f"Слово '{word}' не найдено в словаре")

In [42]:
# Найдите слова, похожие на "духовка"
print(find_synonym('духовка'))

# Найдите слова, похожие на "овощи"
print(find_synonym('овощи'))


Слова, похожие на 'духовка':
ингредиенты: 0.3199
десерт: 0.3064
холодильник: 0.2705
питание: 0.2243
пирог: 0.2142
None

Слова, похожие на 'овощи':
мариновать: 0.2716
хлеб: 0.2691
гриль: 0.2546
фольга: 0.2409
сахар: 0.2108
None


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [44]:
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [45]:
# изменим функцию для новой модели
def ft_model_synonym(word):
  try:
    similar = ft_model.wv.most_similar(word, topn=5)
    print(f"\nСлова, похожие на '{word}':")
    for synonym, score in similar:
        print(f"{synonym}: {score:.4f}")
  except KeyError:
    print(f"Слово '{word}' не найдено в словаре")

In [46]:
# Найдите слова, похожие на "варить"
print(find_synonym('варить'))

# Найдите слова, похожие на "духовка"
print(find_synonym('духовка'))

# Найдите слова, похожие на "овощи"
print(find_synonym('овощи'))


Слова, похожие на 'варить':
вино: 0.2398
ингредиенты: 0.2172
хлеб: 0.1938
брокколи: 0.1846
кипятить: 0.1711
None

Слова, похожие на 'духовка':
ингредиенты: 0.3199
десерт: 0.3064
холодильник: 0.2705
питание: 0.2243
пирог: 0.2142
None

Слова, похожие на 'овощи':
мариновать: 0.2716
хлеб: 0.2691
гриль: 0.2546
фольга: 0.2409
сахар: 0.2108
None


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [47]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Сравниваем для разных слов
compare_models('learning')
compare_models('neural')


Сравнение для слова: 'learning'
  Word2Vec: слово не найдено
  FastText: ['духовка', 'пирог']

Сравнение для слова: 'neural'
  Word2Vec: слово не найдено
  FastText: ['мука', 'травы']


In [48]:
# сравнение FastText и Word2Vec на 3 словах с опечатками
compare_models('shol')
compare_models('пивет')
compare_models('трочка')


Сравнение для слова: 'shol'
  Word2Vec: слово не найдено
  FastText: ['яичница', 'запекать']

Сравнение для слова: 'пивет'
  Word2Vec: слово не найдено
  FastText: ['чашка', 'говядина']

Сравнение для слова: 'трочка'
  Word2Vec: слово не найдено
  FastText: ['травы', 'завтрак']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [49]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [50]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [51]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [52]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [53]:
# cравниваем схожесть doc_2 и doc_4
compare_documents(2, 4)

Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [56]:
# находим самый похожий документ
similar_doc = doc_model.dv.most_similar("doc_1", topn=1)[0]
doc_tag, similarity = similar_doc
doc_id = int(doc_tag.split('_')[1])
print(f"Самый похожий документ на doc_1:")
print(f"  {doc_tag}: {similarity:.4f}")
print(f"  Текст: {documents[doc_id]}")

Самый похожий документ на doc_1:
  doc_0: 0.2735
  Текст: machine learning is interesting


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [57]:
# я выбрала модель FastText

# модель FastText с размерностью 10
ft_model_10 = FastText(
    sentences=cooking_sentences,
    vector_size=10,
    window=3,
    min_count=1,
    workers=2
)

# модель FastText с размерностью 50
ft_model_50 = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

# модель FastText с размерностью 100
ft_model_100 = FastText(
    sentences=cooking_sentences,
    vector_size=100,
    window=3,
    min_count=1,
    workers=2
)

In [61]:
# усовершенствуем функцию из пункта 4 для сравнения модели FastText с разной размерностью
def FastText_find_words(model, dimension, word):
  print(f"\nМодель FastText с размерностью {dimension}")
  try:
    similar = model.wv.most_similar(word, topn=5)
    print(f"\nСлова, похожие на '{word}':")
    for synonym, score in similar:
        print(f"{synonym}: {score:.4f}")
  except KeyError:
    print(f"Слово '{word}' не найдено в словаре")

In [62]:
# применим функцию ко всем 3 моделям на примере 3 слов из пункта 4: духовка, овощи, варить

models = [ft_model_10, ft_model_50, ft_model_100]
words = ['духовка', 'овощи', 'варить']
dimensions = [10, 50, 100]

#сначала написала 3 вложенных цикла, но потом спросила у ии как можно сократить, показал такой вариант
from itertools import product
for model, word, dimension in product(models, words, dimensions):
    print(FastText_find_words(model, dimension, word))


Модель FastText с размерностью 10

Слова, похожие на 'духовка':
фольга: 0.6091
тост: 0.6021
парить: 0.5900
мука: 0.5387
суп: 0.5356
None

Модель FastText с размерностью 50

Слова, похожие на 'духовка':
фольга: 0.6091
тост: 0.6021
парить: 0.5900
мука: 0.5387
суп: 0.5356
None

Модель FastText с размерностью 100

Слова, похожие на 'духовка':
фольга: 0.6091
тост: 0.6021
парить: 0.5900
мука: 0.5387
суп: 0.5356
None

Модель FastText с размерностью 10

Слова, похожие на 'овощи':
парить: 0.7486
лимон: 0.6758
питание: 0.5705
гриль: 0.5695
духовка: 0.5286
None

Модель FastText с размерностью 50

Слова, похожие на 'овощи':
парить: 0.7486
лимон: 0.6758
питание: 0.5705
гриль: 0.5695
духовка: 0.5286
None

Модель FastText с размерностью 100

Слова, похожие на 'овощи':
парить: 0.7486
лимон: 0.6758
питание: 0.5705
гриль: 0.5695
духовка: 0.5286
None

Модель FastText с размерностью 10

Слова, похожие на 'варить':
готовить: 0.7547
парить: 0.7291
травы: 0.6609
мука: 0.6586
взбивать: 0.4311
None

Модель Fa

Для данного датасета и выбранных слов размерность векторов у модели FastText никак не повлиала на результаты. Так, топ-5 слов для всех моделей одинаковы.